In [26]:
import numpy 
import pandas as pd

import requests
from datetime import datetime, timezone
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
from base64 import b64encode
import time
import PIconnect as PI
from PIconnect.PIConsts import UpdateMode, BufferMode

import warnings
warnings.filterwarnings("ignore")

import PIconnect as PI
print(list(PI.PIServer.servers.keys()))

ROOT_URL = "https://127.0.0.1/piwebapi"
COMPUTER = "WIN-5MGCA5NMS7S"
DATABASE = "Database1"
USERNAME = f"{COMPUTER}\Administrator"
PASSWORD = "abcd1234.."

def basic_auth(username, password):
    token = b64encode(f"{username}:{password}".encode('utf-8')).decode("ascii")
    return f'Basic {token}'

BASIC_AUTH_TOKEN = basic_auth(USERNAME, PASSWORD)
POSTHEADERS = { 'X-Requested-With': 'True', 'Content-Type': 'application/json', 'Authorization': BASIC_AUTH_TOKEN }

['WIN-5MGCA5NMS7S']


In [27]:
# Test Server
response_server = requests.get(f"{ROOT_URL}/", auth=(USERNAME, PASSWORD), verify=False)
response_server.json()

{'Links': {'Self': 'https://127.0.0.1/piwebapi/',
  'AssetServers': 'https://127.0.0.1/piwebapi/assetservers',
  'DataServers': 'https://127.0.0.1/piwebapi/dataservers',
  'Omf': 'https://127.0.0.1/piwebapi/omf',
  'System': 'https://127.0.0.1/piwebapi/system'}}

In [28]:
# Get Database WebID
elementPath = f"\\\\{COMPUTER}\\{DATABASE}"
rsp_element_raw = requests.get(f"{ROOT_URL}/assetdatabases?path={elementPath}", auth=(USERNAME, PASSWORD), verify=False)
DATABASE_WEBID = rsp_element_raw.json()['WebId']

# Get DATASERVER WebID
elementPath = f"\\\\{COMPUTER}\\{DATABASE}"
rsp_element_raw = requests.get(f"{ROOT_URL}/dataservers", auth=(USERNAME, PASSWORD), verify=False)
DATASERVER_WEBID = rsp_element_raw.json()['Items'][0]['WebId']

# Get By Path Element
elementPath = f"\\\\{COMPUTER}\\{DATABASE}\\Data_Site\\LGS"
rsp_element_raw = requests.get(f"{ROOT_URL}/elements?path={elementPath}", auth=(USERNAME, PASSWORD), verify=False)
rsp_element_json = rsp_element_raw.json()
ELEMENT_TARGET = rsp_element_json['WebId']

In [39]:
df = pd.read_excel('LGS#1_Overall 2017-2023 (hourly).xlsx', "LGS#1 2023")
df.head()

,TimeStamp,Grid (Aux: 0 & FCE:1),Active Power (MW),Reactive Power (MVAr),Exc Voltage (Volt),Exc Current (A),V Phase 1 (Volt),V Phase 2 (Volt),V Phase 3 (Volt),Current Phase 1 (A),...,LGB Metal Temp 2 (Celcius),LGB Oil Temp (Celcius),Gen air Cooler Air Otlet Temp (Celcius),Gen air Cooler Water Inlet Temp (Celcius),Gen air Cooler Water Otlet Temp (Celcius),Gen Cooler Water Flow (LPM),UGB Cooling Water Flow (LPM),LGB Cooling Water Flow (LPM),Lower Canal Level (msAL),TailRace Level (msAL)
0,2023-01-01 00:00:01,0.0,37.61,19.07,123.82,702.02,10.77,10.73,10.84,0.0,...,53.0,70.0,55.14,61.01,59.31,3795.33,32.42,945.70,308.1,165.96
1,2023-01-01 01:00:01,0.0,37.51,19.04,123.86,704.99,10.77,10.73,10.83,0.0,...,53.0,70.0,55.26,61.63,60.00,3797.60,32.42,946.46,308.1,165.97
2,2023-01-01 02:00:01,0.0,37.34,19.00,123.34,702.22,10.76,10.72,10.83,0.0,...,53.0,70.0,55.39,62.00,60.00,3793.92,32.42,944.96,308.1,165.98
3,2023-01-01 03:00:02,0.0,37.17,18.97,124.49,703.04,10.76,10.72,10.82,0.0,...,53.0,70.0,55.51,62.00,60.00,3798.31,32.42,947.38,308.1,165.99
4,2023-01-01 04:00:02,0.0,37.01,18.93,121.03,702.86,10.75,10.72,10.82,0.0,...,53.0,70.0,55.64,62.00,60.00,3806.77,32.42,948.19,308.1,166.00


In [40]:
df = df.sort_values(by='TimeStamp')

In [41]:
timestamp_df = pd.to_datetime(df.iloc[:,0])
grid_df = df.iloc[:,1] 
insert_df = df.iloc[:,2:].fillna(-5)

timestamp_df

0      2023-01-01 00:00:01
1      2023-01-01 01:00:01
2      2023-01-01 02:00:01
3      2023-01-01 03:00:02
4      2023-01-01 04:00:02
               ...        
7290   2023-10-31 19:59:57
7291   2023-10-31 20:59:57
7292   2023-10-31 21:59:58
7293   2023-10-31 22:59:58
7294   2023-10-31 23:59:59
Name: TimeStamp, Length: 7295, dtype: datetime64[ns]

In [29]:
for series_name, series in insert_df.items():
    name_data = series_name.replace(")", "").split(" (")[0]
    uom = series_name.replace(")", "").split("(")[1]
    series = series.astype('float32')
    print(series_name)
    
    rsp_do_point_raw = requests.post(f"{ROOT_URL}/dataservers/{DATASERVER_WEBID}/points", headers=POSTHEADERS, data=json.dumps({
        "Name": "LGS_" + name_data,
        "Descriptor": "LGS " + name_data,
        "PointClass": "classic",
        "PointType": "float32",
        "EngineeringUnits": "",
        "Step": False,
        "Future": True,
        "DisplayDigits": -5
    }), verify=False)
    #print(rsp_do_point_raw)

    rsp_do_raw = requests.post(f"{ROOT_URL}/elements/{ELEMENT_TARGET}/attributes", headers=POSTHEADERS, data=json.dumps({
        "Name": name_data,
        "Description": name_data,
        "Type": "Double",
        "TypeQualifier": "",
        "DefaultUnitsName": '',
        "DisplayDigits": -5,
        "DataReferencePlugIn": "PI Point",
        "ConfigString": "LGS_" + name_data,
        "IsConfigurationItem": False,
        "IsHidden": False,
        "IsManualDataEntry": False
    }), verify=False)
    print(rsp_do_raw)
    time.sleep(1)
    

Active Power (MW)
Reactive Power (MVAr)
Exc Voltage (Volt)
Exc Current (A)
V Phase 1 (Volt)
V Phase 2 (Volt)
V Phase 3 (Volt)
Current Phase 1 (A)
Current Phase 2 (A)
Current Phase 3 (A)
Modbus (MVAr)
Modbus (MVA)
PF (Cos Phi)
Freq (hz)
Speed (RPM)
WG Opening (%)
Speed (RPM).1
Penstock Pressure (Kg/cm2)
Penstoc Flow (Cumecs)
Turbine Flow (Cumecs)
Eff (Cumecs/MW)
UGB X (µm)
UGB Y (µm)
LGB X (µm)
LGB Y (µm)
TGB X (µm)
TGB Y (µm)
Stator Winding temp 13 (Celcius)
Stator Winding temp 14 (Celcius)
Stator Winding temp 15 (Celcius)
Stator Core temp 12  (Celcius)
UGB Metal Temp  (Celcius)
UGB Oil Temp  (Celcius)
LGB Metal Temp 1 (Celcius)
LGB Metal Temp 2 (Celcius)
LGB Oil Temp (Celcius)
Gen air Cooler Air Otlet Temp (Celcius)
Gen air Cooler Water Inlet Temp (Celcius)
Gen air Cooler Water Otlet Temp (Celcius)
Gen Cooler Water Flow (LPM)
UGB Cooling Water Flow (LPM)
LGB Cooling Water Flow (LPM)
Lower Canal Level (msAL)
TailRace Level (msAL)


In [42]:
for series_name, series in insert_df.items():
    name_data = series_name.replace(")", "").split(" (")[0]
    uom = series_name.replace(")", "").split("(")[1]
    series = series.astype('float32')
    current_point_name = "LGS_" + name_data
    print(series_name)

    with PI.PIServer(server=COMPUTER) as server:
        point = server.search(current_point_name)[0]
        for index, value in series.items():
            point.update_value(
                    value,
                    timestamp_df[index].to_pydatetime(),
                    UpdateMode.NO_REPLACE,
                    BufferMode.DO_NOT_BUFFER,
                )

            # time.sleep(50/1000)
            # print(index / len(series))

Active Power (MW)
Reactive Power (MVAr)
Exc Voltage (Volt)
Exc Current (A)
V Phase 1 (Volt)
V Phase 2 (Volt)
V Phase 3 (Volt)
Current Phase 1 (A)
Current Phase 2 (A)
Current Phase 3 (A)
Modbus (MVAr)
Modbus (MVA)
PF (Cos Phi)
Freq (hz)
Speed (RPM)
WG Opening (%)
Speed (RPM).1
Penstock Pressure (Kg/cm2)


In [ ]:
# Row 1683 Bermassaslah Date nya